In [ ]:
import os, sys
sys.path.append("/content")
from src.utils import fit_mlp_cv, plot_obs_predict

with open("/content/credentials") as f:
    env_vars = f.read().split("\n")

for var in env_vars:
    key, value = var.split(" = ")
    os.environ[key] = value

storage_options = {"account_name":os.environ["ACCOUNT_NAME"],
                   "account_key":os.environ["BLOB_KEY"]}


In [ ]:
import numpy as np
import pandas as pd
import torch.nn as nn
from sklearn.metrics import r2_score

# Specify hyperparameters and fit model
buffer_distance = 500
day_tolerance = 8
cloud_thr = 80
features = [
    "sentinel-2-l2a_AOT", 
    "sentinel-2-l2a_B02", "sentinel-2-l2a_B03", "sentinel-2-l2a_B04", # RGB bands
    "sentinel-2-l2a_B08", # NIR
    #"sentinel-2-l2a_WVP", 
    "sentinel-2-l2a_B05", "sentinel-2-l2a_B06", "sentinel-2-l2a_B07", "sentinel-2-l2a_B8A",  # Red edge bands
    # "is_brazil", #"sine_julian", 
    "sentinel-2-l2a_B11", "sentinel-2-l2a_B12", # SWIR
    # "mean_viewing_azimuth", "mean_viewing_zenith",
    # "mean_solar_azimuth", "mean_solar_zenith"
]
epochs = 500
batch_size = 32
learning_rate = 0.001

layer_out_neurons = [12, 16, 8]

mask_method1 = "lulc"
mask_method2 = "mndwi"
min_water_pixels = 20
learn_sched_step_size = 250 
learn_sched_gamma = 0.5
activation_function = nn.SELU()

In [ ]:
output = fit_mlp_cv(
    features,
    learning_rate,
    batch_size,
    epochs,
    storage_options=storage_options,
    activation_function=activation_function,
    buffer_distance=buffer_distance,
    day_tolerance=day_tolerance,
    cloud_thr=cloud_thr,
    mask_method1=mask_method1,
    mask_method2=mask_method2,
    min_water_pixels=min_water_pixels,
    layer_out_neurons=layer_out_neurons,
    learn_sched_step_size=learn_sched_step_size,
    learn_sched_gamma=learn_sched_gamma,
    verbose=True
)

In [ ]:
part = "validation"
my_fold = 2

if part == "validation":
    pred_fold = output["y_pred_val_fold"]
    y_fold = output["y_obs_val_fold"]
else:
    pred_fold = output["y_pred_train_fold"]
    y_fold = output["y_obs_train_fold"]

print(f"R2: {r2_score(pred_fold[my_fold], y_fold[my_fold])}")
plot_obs_predict(pd.DataFrame({"pred": pred_fold[my_fold], "obs": y_fold[my_fold]}), f"Observed vs. Predicted, Fold {my_fold+1}")

# pd.DataFrame({
#     "site_no": list(data[data["fold_idx"] == my_fold + 1]["site_no"]),
#     "se": (np.array(y_fold[my_fold]) - np.array(pred_fold[my_fold])) ** 2
# })
# list(data[data["fold_idx"] == my_fold + 1]["site_no"])

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(8,6))
plt.plot(output["train_loss_fold"][my_fold], color="teal", label="training")
plt.plot(output["val_pooled_loss_fold"][my_fold], color="orange", label="validation (pooled)")
plt.plot(output["val_site_loss_fold"][my_fold], color="purple", label="validation (site mean)")
plt.title(f"Loss Curves for Training and Validation Data, Fold {my_fold + 1}")
plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.legend()
plt.show()
